In [50]:
import pandas as pd
import numpy as np
import os
import ast
import glob
import re

#explode=False
#all_registers=False


In [51]:
#! cat HI_with_nvidia.jsonl | head -n 10 > test_HI_with_nvidia.jsonl

In [52]:
datas = []

labels_all_hierarchy_with_other = {
    "MT": ["MT"],
    "LY": ["LY"],
    "SP": ["SP", "it", "os"],
    "ID": ["ID"],
    "NA": ["NA", "ne", "sr", "nb", "on"],
    "HI": ["HI", "re", "oh"],
    "IN": ["IN", "en", "ra", "dtp", "fi", "lt", "oi"],
    "OP": ["OP", "rv", "ob", "rs", "av", "oo"],
    "IP": ["IP", "ds", "ed", "oe"],
}

final_scores = {"MT": 0.374349,
    "LY": 0.357626,
    "SP": 0.422314,
    "ID": 0.430686,
    "NA": 0.441469,
    "HI": 0.446558,
    "IN": 0.437347,
    "OP":  0.446629,
    "IP": 0.393026,
    "HI-IN":0.464502,
    "ne": 0.417933,
    "dtp": 0.452347,
    }

with os.scandir("result") as f:
    for file in f:
        filename = file.name
        print(f"reading {filename}")
        with open("result/"+filename) as f:
            data = []
            for line in f:
                line = re.sub(r"tensor\((\[.*?\])(?:, device='.*?')?\)", r"\1", line)
                try:
                    data.append(ast.literal_eval(line))
                except:
                    print("error:", line)
        df_ = pd.DataFrame(data)
        df_ = df_.drop(columns="text")
        df_["parsed_register"] = df_["register"].map(lambda reg: [r for r in reg if r in list(labels_all_hierarchy_with_other.keys())+["ne", "dtp"]])
        df_["parsed_register"] = df_["parsed_register"].map(lambda reg: "-".join(reg))
        df_["parsed_register"] = df_["parsed_register"].map(lambda reg: {"NA-ne":"ne", "ne-NA": "ne", "IN-dtp":"dtp", "dtp-IN":"dtp", "IN-HI":"HI-IN"}.get(reg, reg))
        df_["final_score"] = df_["parsed_register"].map(lambda reg: final_scores[reg])
        df_["predicted_class"] = df_["predicted_class"].map(lambda x: int(x[0]))
        datas.append(df_)



df = pd.concat(datas)

reading MT_with_nvidia.jsonl
reading OP_with_nvidia.jsonl
reading HI-IN_with_nvidia.jsonl
reading dtp_with_nvidia.jsonl
reading IN_with_nvidia.jsonl
reading LY_with_nvidia.jsonl
reading ID_with_nvidia.jsonl
reading HI_with_nvidia.jsonl
reading ne_with_nvidia.jsonl
reading NA_with_nvidia.jsonl
reading SP_with_nvidia.jsonl
reading IP_with_nvidia.jsonl


In [53]:
display(df)
print(np.unique(df["parsed_register"].tolist(), return_counts=True))

,id,register,predicted_class,predicted_domain,parsed_register,final_score
0,aa6e226e50524b13656a48cca50ef88b,[MT],2,[Low],MT,0.374349
1,48ea34d9aa0ce1a6ac031ee3ee950eda,[MT],2,[Low],MT,0.374349
2,7d8fda8eca0b17ff6faab5f5837356dd,[MT],2,[Low],MT,0.374349
3,820661e03daf8d1268772cd7a0c84856,[MT],2,[Low],MT,0.374349
4,56831efa9c0dbe62578889db0c02cd2d,[MT],2,[Low],MT,0.374349
...,...,...,...,...,...,...
9995,67a9f23998d99875ee1f81a65a2d2310,"[ds, IP]",2,[Low],IP,0.393026
9996,e15363603d2ad016a5073e0cd89417a6,"[IP, ds]",1,[Medium],IP,0.393026
9997,587640cea60f50a8c220c04dc2ff5dca,[IP],1,[Medium],IP,0.393026
9998,3ea3bcc66d48f1cf2814bae9be0222f0,"[ds, IP]",0,[High],IP,0.393026


(array(['HI', 'HI-IN', 'ID', 'IN', 'IP', 'LY', 'MT', 'NA', 'OP', 'SP',
       'dtp', 'ne'], dtype='<U5'), array([10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000,
       10000, 10000, 10000]))


In [54]:
def cooccurrence_matrix(df):
    regs = np.unique(df["parsed_register"]).tolist()
    gens = np.unique(df["predicted_class"]).tolist()
    print("regs: ", regs)
    print("gens: ", gens)

    print(range(len(regs)))
    reg2index = {k:v for k,v in zip(regs, range(len(regs)))}
    gen2index = {k:v for k,v in zip(gens, range(len(gens)))}

    m = np.zeros((len(regs), len(gens)), dtype=np.float32)
    summa = 0
    for i,d in df.iterrows():
        r = d["parsed_register"]
        g = d["predicted_class"]

        m[reg2index[r],gen2index[g]] += 1
        summa+=1

    return m/summa


m = cooccurrence_matrix(df)

assert 0.999 < sum(m.sum(axis=0)) < 1.001 and 0.999 < sum(m.sum(axis=1)) < 1.001

regs:  ['HI', 'HI-IN', 'ID', 'IN', 'IP', 'LY', 'MT', 'NA', 'OP', 'SP', 'dtp', 'ne']
gens:  [0, 1, 2]
range(0, 12)


In [55]:
print(sum(m.sum(axis=0)))
print(sum(m.sum(axis=1)))

1.0
1.0000000000000002


In [56]:
def entropy(m):
    p_gen = m.sum(axis=0)
    p_reg = m.sum(axis=1)

    def _entropy_1d(v):
        return -1*sum(v*np.log2(v))

    def _entropy_2d(m):
        ent = 0
        for j in m:
            for k in j:
                if k != 0:
                    ent += k*np.log2(k)
        return -1*ent

    ent_reg = _entropy_1d(p_reg)
    ent_gen = _entropy_1d(p_gen)
    ent_reg_gen = _entropy_2d(m)

    return ent_reg, ent_gen, ent_reg_gen


def mutual_info(m):
    reg_pdf = m.sum(axis=1)
    gen_pdf = m.sum(axis=0)

    info = 0
    for i in range(len(reg_pdf)):
        for j in range(len(gen_pdf)):
            if m[i,j]!= 0:
                info += m[i,j]*np.log(m[i,j]/(reg_pdf[i]*gen_pdf[j]))
    return info


print(entropy(m))
print(mutual_info(m))

(3.5849625007211565, 1.3793877445248428, 4.676861404436686)
0.19927207944942177


In [57]:
from scipy.stats import chi2_contingency

def cramers_v(df):
    x = df["parsed_register"]
    y = df["predicted_class"]

    confusion_matrix = pd.crosstab(x, y)
    print(confusion_matrix)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))
print(cramers_v(df))

predicted_class     0     1     2
parsed_register                  
HI               1856  7107  1037
HI-IN            1881  6637  1482
ID                 16  4087  5897
IN               1337  5612  3051
IP                559  4775  4666
LY                120  5033  4847
MT                  0   589  9411
NA               1174  7231  1595
OP               1243  7813   944
SP               2198  7447   355
dtp              1830  6138  2032
ne               4183  5195   622
0.43817847177510694


In [58]:
! pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [59]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Assuming 'nominal_var' is your nominal column and 'ordinal_var' is numeric (1 to 5)
model = ols('predicted_class ~ C(parsed_register)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
eta_squared = anova_table['sum_sq']['C(parsed_register)'] / anova_table['sum_sq'].sum()
print(eta_squared)

0.30657679613860916


In [60]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [61]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encode feature1
df['feature1_encoded'] = le.fit_transform(df['parsed_register'])

# Encode feature2
df['feature2_encoded'] = le.fit_transform(df['predicted_class'])

# Extract the encoded features
X = df[['feature1_encoded']]
y = df['feature2_encoded']

# Calculate mutual information
mi = mutual_info_classif(X, y, discrete_features=True)

print(f'Mutual Information between feature1 and feature2: {mi[0]}')

Mutual Information between feature1 and feature2: 0.19927207944942277


In [62]:
! pip install plotly
! pip install nbformat>=4.2.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [63]:
import pandas as pd
import plotly.graph_objects as go

# Example: sample data
# df = pd.DataFrame({
#     'parsed_register': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'C', 'A'],
#     'int_score': [1, 2, 2, 3, 3, 4, 1, 5, 4, 2]
# })

# Count pairs (grouped by nominal and ordinal labels)
df['predicted_class'] = df['predicted_class'].astype(str)  # make sure it's a string for plotting
counts = df.groupby(['parsed_register', 'predicted_class']).size().reset_index(name='count')

# Get all unique labels
all_labels = list(pd.unique(counts['parsed_register'].tolist() + counts['predicted_class'].tolist()))
label_to_index = {label: i for i, label in enumerate(all_labels)}

# Build Sankey inputs
source_indices = counts['parsed_register'].map(label_to_index)
target_indices = counts['predicted_class'].map(label_to_index)
values = counts['count']

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_labels,
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="Sankey Diagram: Register vs NVIDIA score", font_size=12, width=600)
fig.show()
